### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
## Load the dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
## Split the data into training, validation and testset

train, test = train_test_split(data, test_size = 0.15, random_state = 42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1235,7.8,0.32,0.31,1.70,0.036,46.0,195.0,0.99300,3.03,0.48,10.5,5
1108,6.1,0.29,0.27,1.70,0.024,13.0,76.0,0.98930,3.21,0.51,12.6,7
3945,6.2,0.27,0.32,6.30,0.048,47.0,159.0,0.99282,3.21,0.60,11.0,6
3066,7.4,0.21,0.80,12.30,0.038,77.0,183.0,0.99778,2.95,0.48,9.0,5
2960,7.7,0.11,0.34,14.05,0.040,41.0,114.0,0.99634,3.07,0.59,11.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [4]:
train_x = train.drop(["quality"], axis = 1).values
train_y = train[["quality"]].values.ravel() # ravel will make 1 dim

## Validation dataset
test_x = test.drop(["quality"], axis = 1).values
test_y = test[["quality"]].values.ravel()

In [5]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x,train_y, test_size=0.15, random_state=42)

signature =  infer_signature(train_x, train_y)

In [6]:
### ANN model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x,test_y):

    ## Define model architechture
    mean = np.mean(train_x, axis = 0) # axis = 0 means column with mean
    var = np.var(train_x, axis = 0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    ## Compile the model
    model.compile(
        optimizer = keras.optimizers.SGD(learning_rate=params["lr"],
                                         momentum=params["momentum"]),
        loss = "mean_squared_error",
        metrics = [keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with MLFLOW tracking
    with mlflow.start_run(nested = True):
        model.fit(
            train_x,
            train_y,
            validation_data = (valid_x,valid_y),
            epochs = epochs,
            batch_size = 64
        )

        ## Evaluate the model
        eval_result = model.evaluate(valid_x,valid_y, batch_size = 64)

        eval_rmse = eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("Eval_rmse", eval_rmse)

        ## Log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [7]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x = train_x,
        train_y=train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y=test_y
    )
    return result

In [8]:
space = {
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum": hp.uniform("momentum",0.0,0.1)
}

In [10]:
mlflow.set_experiment("/wine-quality")

with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo=tpe.suggest,
        max_evals = 4,
        trials = trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key = lambda x : x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")
    

2025/05/13 02:47:16 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/56 ━━━━━━━━━━━━━━━━━━━━ 53s 980ms/step - loss: 30.9320 - root_mean_squared_error: 5.5617
22/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.7009 - root_mean_squared_error: 4.0151   
48/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11.3409 - root_mean_squared_error: 3.2493
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 10.3446 - root_mean_squared_error: 3.0904 - val_loss: 1.6072 - val_root_mean_squared_error: 1.2678

Epoch 2/3                                            

 1/56 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 1.2381 - root_mean_squared_error: 1.1127
27/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3225 - root_mean_squared_error: 1.1498 
55/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3123 - root_mean_squared_error: 1.1454
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3109 - root_mean_squared_error: 1.1448 - val_loss: 1.2265 - val_root_mean_squared_error: 1.1075

Epoch 3/3                                            

 1/

In [12]:
## Inferancing

import mlflow

model_uri = 'runs:/d2146b3db089413eabbba262dd9ba515/model'

# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
input_data = test_x

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="local",
)

2025/05/13 02:57:09 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
2025/05/13 02:57:09 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
{"predictions": [[5.330509185791016], [6.678794860839844], [6.3922929763793945], [4.972773551940918], [5.678166389465332], [6.777868270874023], [5.852817535400391], [5.625035285949707], [6.442533493041992], [6.591113090515137], [6.954345703125], [4.547308921813965], [6.8898468017578125], [4.770693778991699], [6.216418266296387], [5.046977996826172], [7.3248186111450195], [6.379878997802734], [6.0762128829956055], [5.519220352172852], [5.537153244018555], [6.552518844604492], [4.591621398925781], [5.391119956970215], [5.284780502319336], [4.739828109741211], [4.624824047088623], [6.078740119934082], [5.892329216003418], [4.515091419219971], [5.786224365234375], [5.136102676391602], [6.594532012939453], [4.903541088104248], [5.775080680847168], [6.226853370666504], [7.061994552612305], [5.164926528930664], [5.534799575805664], [5.990505218505859], [5.5212907791137695], [4.982534408569336], [5.311834335327148], [5.597153663635254], [4.69978141784668]